In [9]:
from ase.build import fcc100, add_adsorbate, bulk
from ase.io import write
import ase
from catkit.gen.adsorption import get_adsorption_sites
import catkit
import numpy as np


# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
vacuum = 15.0
slab = fcc100('Cu', size=(4,4,4), a=Cu_alat, vacuum=vacuum)
write('Cu_slab.cif', slab)

# try build slab using ASE -- no option for full set (not sym reduced) set of adsorption sites
free_slab = slab.copy()
# possible sites are ‘ontop’, ‘hollow’, ‘hollow’.
add_adsorbate(free_slab, 'Ag', 0, 'hollow')
write('hollow_slab.cif', free_slab)

# free_slab.get_surface_atoms()

# free_slab_gratoms = catkit.gratoms.Gratoms(free_slab)
# get_adsorption_sites(free_slab_gratoms)

# write('abs_slab.cif', abs_slab)
# slab.center(vacuum=10.0, axis=2)

In [13]:
type(catkit_slab) is catkit.gratoms.Gratoms or ase.Atoms

True

In [21]:
def get_random_idx(connectivity, type=None):
    """Get random site index
    """
    connectivities = {
        'top': 1,
        'bridge': 2,
        'hollow': 4
    } # defaults to hollow

    # top should have connectivity 1, bridge should be 2 and hollow more like 4

    if type:
        # TODO: implemente preferred connecitivity
        pass
    
    rand_idx = random.randrange(len(connectivity))

    return rand_idx

In [22]:
# build surface using catkit instead
a1 = bulk('Cu', 'fcc', a=Cu_alat)
catkit_slab = catkit.build.surface(a1, size=(4,4,4), miller=(1,0,0), termination=0, fixed=0, vacuum=15, orthogonal=False)
write('gratoms.cif', catkit_slab)
# written out .cif file looks good

#SlabGenerator-> layer_type='angs', layers=10...

# get ALL the adsorption sites
# top should have connectivity 1, bridge should be 2 and hollow more like 4
coords, connectivity, sym_idx = get_adsorption_sites(catkit_slab, symmetry_reduced=False)
print(f"In pristine slab, there are a total of {len(connectivity)} sites")

# adsorb to a random site
import random
random.seed(7)
rand_idx = get_random_idx(connectivity)
# rand_idx = len(coords)-1
rand_site = coords[rand_idx]
print(f"idx is {rand_idx} at {rand_site} with connectivity {connectivity[rand_idx]}")

Ag = catkit.gratoms.Gratoms('Ag')
# tag the atom to be adsorbed with its to-be index (last position on slab)    
adsorbate_idx = len(catkit_slab)
Ag.set_tags(adsorbate_idx)
catkit_slab_build = catkit.gen.adsorption.Builder(catkit_slab)
catkit_slab_w_adsorbate = catkit_slab_build.add_adsorbate(Ag, bonds=[0], index=rand_idx, auto_construct=False, symmetric=False)
write(f'gratoms_adsorbed.cif', catkit_slab_w_adsorbate)

# get adsorption sites for catkit_slab_w_adsorbate
_, new_connectivity, _ = get_adsorption_sites(catkit_slab_w_adsorbate, symmetry_reduced=False)
print(f"In adsorbed slab, there are a total of {len(new_connectivity)} sites") # which is the same as the number in pristine slab

# try to adsorb to the same site using another atom -- worked, still the same atom
Na = catkit.gratoms.Gratoms('Na')
# Na.set_tags(rand_idx)
catkit_slab_ads_build = catkit.gen.adsorption.Builder(catkit_slab_w_adsorbate)
catkit_slab_again_adsorbate = catkit_slab_ads_build.add_adsorbate(Na, bonds=[0], index=rand_idx, auto_construct=False, symmetric=False)
write(f'gratoms_again_adsorbed.cif', catkit_slab_again_adsorbate)

In pristine slab, there are a total of 64 sites
idx is 41 at [ 6.3899472   7.66793665 20.42205   ] with connectivity 2
In adsorbed slab, there are a total of 64 sites


In [15]:
# try setting tag
slab_tags = catkit_slab_w_adsorbate.get_tags()

assert len(np.argwhere(slab_tags==adsorbate_idx)) <= 1, "more than 1 site found"
assert len(np.argwhere(slab_tags==adsorbate_idx)) == 1, "no sites found"
np.argwhere(slab_tags==adsorbate_idx)[0,0]

# remove adsorbate
gatoms_for_del = catkit_slab_w_adsorbate.copy()
del gatoms_for_del[adsorbate_idx]

# assert slab after deletion is the same as slab before adsorption
print(gatoms_for_del == catkit_slab)
write(f"gratoms_deleted.cif", gatoms_for_del)

True


In [4]:
slab_tags

array([ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 64])

In [5]:
len(catkit_slab)

64

In [6]:
from catkit.gen.surface import SlabGenerator
from ase.visualize import view

# Make a test slab
atoms = bulk('Pd', 'fcc', cubic=True)
atoms[3].symbol = 'Cu'

gen = SlabGenerator(
    atoms,
    miller_index=(2, 1, 1),
    layers=9,
    fixed=5,
    vacuum=4)
free_slab.info

/home/dux/.local/lib/python3.9/site-packages/catkit/gen/surface.py:105: UserWarning: Not using a standardized bulk will result in an arbitrary Miller index. To get ensure you are using the correct miller index, use standardize_bulk=True
  warnings.warn(


{'adsorbate_info': {'cell': array([[2.55597888, 0.        ],
         [0.        , 2.55597888]]),
  'sites': {'ontop': (0, 0), 'hollow': (0.5, 0.5), 'bridge': (0.5, 0)},
  'top layer atom index': 48}}